<a href="https://colab.research.google.com/github/pavolbauer/lindoscope-2021/blob/main/labs/W1D3_Behavior_Analysis/W1D3_Behavior_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

Welcome to the behavior analysis lab!